In [1]:
import tkinter as tk
from tkinter import StringVar
import socket
import threading
from PIL import ImageTk, Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import struct
import os
import time
HOST = '172.20.10.2'
PORT = 5550
# 创建服务器端套接字
sk = socket.socket()
# 设置给定套接字选项的值。
sk.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
# 把地址绑定到套接字
sk.bind((HOST, PORT))
# 监听链接
sk.listen()
# 接受客户端链接
conn, addr = sk.accept()

window = tk.Tk()

recTime = time.time()

State = []
State.append(0)



def start():
    sendThreadFunc('1')
    State[0] = 1
def sleep():
    sendThreadFunc("2")
    State[0] = 2
def stop():
    sendThreadFunc("4")
    State[0] = 4
def ReceiveImg():
    #if time.time()-recTime < 5:
        #return
    sendThreadFunc("3")
    State[0] = 3
    recTime = time.time()
def HumanControl():
    if State[0]==5:
        State[0]=0
        string.set("Not in Control Mode!")
        sendThreadFunc("8")
        return
    State[0] = 5
    sendThreadFunc("8")
    string.set("Press Key to Control Robot!")


def sendThreadFunc(myword):
    try:
        conn.send(myword.encode())
    except ConnectionAbortedError:
        print('Server closed this connection!')
    except ConnectionResetError:
        print('Server is closed!')

def recvThreadFunc():
    while True:
        try:
            fileinfo_size = struct.calcsize('128sl')
            buf = conn.recv(fileinfo_size)
            if buf:
                filename, filesize = struct.unpack('128sl', buf)
                fn = filename.strip(str.encode('\00'))
                new_filename = os.path.join(str.encode('./'), str.encode('new_') + fn)
                print ('file new name is {0}, filesize if {1}'.format(new_filename, filesize))
                if os.path.exists(new_filename):
                    os.remove(new_filename)
                recvd_size = 0  # 定义已接收文件的大小
                fp = open(new_filename, 'wb')
                print ("start receiving...")
                while not recvd_size == filesize:
                    if filesize - recvd_size > 1024:
                        data = conn.recv(1024)
                        recvd_size += len(data)
                    else:
                        data = conn.recv(filesize - recvd_size)
                        recvd_size = filesize
                    fp.write(data)
                fp.close()
                tempimg = Image.open("C:/Users/YYX/Desktop/new_final.jpg").resize((1100,700))
                tempimg = ImageTk.PhotoImage(image = tempimg)
                imageshow.configure(image = tempimg)
                imageshow.update()
                window.update()
                print ("end receive...")
                
        except ConnectionAbortedError:
            print('Server closed this connection!')
        except ConnectionResetError:
            print('Server is closed!')
            
def up():
    if State[0]!=5:
        return
    string.set("Robot is going UP")
    sendThreadFunc("4")
def down():
    if State[0]!=5:
        return
    string.set("Robot is going DOWN")
    sendThreadFunc("5")
def left():
    if State[0]!=5:
        return
    string.set("Robot is going LEFT")
    sendThreadFunc("6")
def right():
    if State[0]!=5:
        return
    string.set("Robot is going RIGHT")
    sendThreadFunc("7")

    
    
def sendmess(event):
    if State[0] != 5:
        return
    inp = event.keysym
    if inp=='Up':
        StateString = 'UP'
        sendThreadFunc("4")
    elif inp=="Down":
        StateString = 'DOWN'
        sendThreadFunc("5")
    elif inp == "Left":
        StateString = 'LEFT'
        sendThreadFunc("6")
    elif inp == 'Right':
        StateString = 'RIGHT'
        sendThreadFunc("7")
    else:
        return
    string.set("Robot is going "+StateString)


window.title("IP address : "+str(HOST))
window.geometry('1600x1000')

string = StringVar()
controlState = tk.Label(window, textvariable=string, width = 50, height = 5, font=('Arial', 18))
string.set("Not in Control Mode!")
controlState.place(x=1030, y=200)
window.bind("<Key>", sendmess)

title = tk.Label(window, text='Mask Detector Remote Control Pannel', font=('Arial', 24), width=50, height=2)
title.pack()

startB = tk.Button(window, text='Start', font=('Arial', 12), width=20, height=2, command=start)
startB.place(x=100,y=900)
sleepB = tk.Button(window, text='Sleep', font=('Arial', 12), width=20, height=2, command=sleep)
sleepB.place(x=400,y=900)
stopB = tk.Button(window, text='Stop', font=('Arial', 12), width=20, height=2, command=stop)
stopB.place(x=700,y=900)
RecB = tk.Button(window, text='Receive Image', font=('Arial', 12), width=20, height=2, command=ReceiveImg)
RecB.place(x=1000,y=900)
CtrlB = tk.Button(window, text='Control Mode', font=('Arial', 12), width=20, height=2, command=HumanControl)
CtrlB.place(x=1300,y=900)

upB = tk.Button(window, text='FORWARD', font=('Arial', 12), width=10, height=5, command=up)
upB.place(x=1330,y=500)
downB = tk.Button(window, text='BACKWARD', font=('Arial', 12), width=10, height=5, command=down)
downB.place(x=1330,y=650)
leftB = tk.Button(window, text='LEFT', font=('Arial', 12), width=10, height=5, command=left)
leftB.place(x=1200,y=650)
rightB = tk.Button(window, text='RIGHT', font=('Arial', 12), width=10, height=5, command=right)
rightB.place(x=1460,y=650)

InitImage = Image.open("C:/Users/YYX/Desktop/test.jpg").resize((1100, 700))
InitImage = ImageTk.PhotoImage(image = InitImage)
imageshow = tk.Label(window, height=700, width=1100, image=InitImage)
imageshow.place(x=50,y=110)
imageshow.update()

socketThread = threading.Thread(target = recvThreadFunc)
socketThread.start()
# 进入消息循环
window.mainloop()

file new name is b'./new_final.jpg', filesize if 192512
start receiving...
Exception in thread Thread-4:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-f39f230918a7>", line 89, in recvThreadFunc
  File "C:\ProgramData\Anaconda3\lib\site-packages\PIL\Image.py", line 1910, in resize
    self.load()
  File "C:\ProgramData\Anaconda3\lib\site-packages\PIL\ImageFile.py", line 255, in load
    raise OSError(
OSError: image file is truncated (24 bytes not processed)
